In [50]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('train.csv',index_col=0)

In [51]:
import sklearn.linear_model as lm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from neupy import algorithms, environment
from sklearn import metrics

In [52]:
y = dataset.species

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
if y.dtype == object:
    y = le.fit_transform(y)
else:
    pass

In [53]:
X = dataset.drop('species', axis=1)
X = (dataset.ix[:,1:].values).astype('float32')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [54]:
#Splitting data to 5 sets and performing multinomial regression on each
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

log_loss_total = []
accuracy_score = []
i=0
environment.reproducible()
lgr = lm.LogisticRegression(multi_class = 'multinomial', solver = 'newton-cg')
pnn = algorithms.PNN(std=10, verbose=False)

for train_index, test_index in skf.split(X, y):    
    X_train = []
    X_test = []
    X_train, X_test, y_train, y_test = (X[train_index], X[test_index], y[train_index], y[test_index])
    
    # Multinomial linear regression
    lgr.fit(X_train, y_train)
    y_predicted = lgr.predict_proba(X_test)
    logLoss = log_loss(y_test, y_predicted)
    log_loss_total.append(logLoss)
    i += 1
    print("Log loss for fold " + str(i) + "= " + str(logLoss))
    
    #PNN classifier
    pnn.train(X_train, y_train)
    y_predicted_pnn = pnn.predict(X_test)
    accuracy_score.append(metrics.accuracy_score(y_test, y_predicted_pnn))
    
log_loss_mean = np.mean(log_loss_total)
print("Average log loss is " + str(log_loss_mean))
accuracy_score_mean = np.mean(accuracy_score)
print("Average accuracy score using pnn classifier = "+ str(accuracy_score_mean))

Time: 0:00:00 | 50%|######################                      | ETA:  0:00:00

Log loss for fold 1= 4.179430033221389


/opt/conda/lib/python3.6/site-packages/scipy/optimize/linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Log loss for fold 2= 4.1698608723553745


Time: 0:00:00 | 50%|######################                      | ETA:  0:00:00

Log loss for fold 3= 4.181611604160732


Time: 0:00:00 | 50%|######################                      | ETA:  0:00:00

Log loss for fold 4= 4.17900683061041


Time: 0:00:00 | 50%|######################                      | ETA:  0:00:00

Log loss for fold 5= 4.1701799318043875
Average log loss is 4.176017854430459
Average accuracy score using pnn classifier = 0.7585858585858585


In [56]:
#Predicting the value for the test data

test_dataset = pd.read_csv('test.csv',index_col=0)
y_test_dataset = dataset.species

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
if y_test_dataset.dtype == object:
    y_test_dataset = le.fit_transform(y)
else:
    pass

X_test_dataset = dataset.drop('species', axis=1)
X_test_dataset = (dataset.ix[:,1:].values).astype('float32')

y_pred_test = lgr.predict_proba(X_test_dataset)
log_loss_test = log_loss(y_test_dataset,y_pred_test)

y_predicted_test_pnn = pnn.predict(X_test_dataset)
print(metrics.accuracy_score(y_test_dataset, y_predicted_test_pnn))
print("log loss for the test data using Multinomial regression= " + str(log_loss_test))

Time: 0:00:00 | 75%|#################################           | ETA:  0:00:00

0.807070707070707
log loss for the test data using Multinomial regression= 4.155838008601256
